### Preprocess on raw Cornell Data

In [4]:
import pandas as pd
import numpy as np
from imdb import Cinemagoer

import ast

In [ ]:
movies = pd.read_csv(r'C:\Users\98768\Desktop\is310\project\movie-corpus\conversations.csv')
#corpus = pd.read_csv(r'C:\Users\98768\Desktop\is310\project\movie-corpus\corpus.csv')
speakers = pd.read_csv(r'C:\Users\98768\Desktop\is310\project\movie-corpus\speakers.csv')
utterances = pd.read_csv('C:\\Users\\98768\\Desktop\\is310\\project\\movie-corpus\\utterances.csv')

movies['release_year'] = movies['release_year'].str.replace('/I', '')
movies['release_year'] = movies['release_year'].astype(int)

utterances.rename(columns={'id': 'utterances_id'}, inplace=True)
utterances.rename(columns={'conversation_id': 'belonging_id'}, inplace=True)
utterances.rename(columns={'meta.movie_id': 'movie_idx'}, inplace=True)
utterances.rename(columns={'reply-to': 'reply_to'}, inplace=True)
utterances['reply_to'].fillna('end', inplace=True) # fill NaN with 'end' to indicate this utterances is the end of the conversation

speakers['gender'] = speakers['gender'].str.lower() # convert gender to lowercase
speakers['credit_pos'] = speakers['credit_pos'].replace('?', 0) # convert unkwnown credit position = 0
speakers['credit_pos'] = speakers['credit_pos'].astype(int)

print('conversations:', movies.columns.values)
#print('corpus:', corpus.columns.values)
print('speakers:', speakers.columns.values)
print('utterances:', utterances.columns.values)

In [ ]:
ia = Cinemagoer()

# getting the movie id from Cinemagoer
movie_ids = []

for title in movies['movie_name']:
    if ia.search_movie(title) is not None:
        id = ia.search_movie(title)[0].movieID
    else:
        id = None
    movie_ids.append(id)

movies['Cinemagoer_id'] = movie_ids

In [ ]:
# getting the plot (intro) from the Cinemagoer movie id
def get_intro(movie_id):
    if movie_id is not None:
        movie = ia.get_movie(movie_id, info=['plot'])
        if 'intro' in movie:
            print(movie['intro'])
            return movie['intro']
    else:
        return None

# Add the plots to the movies DataFrame
movies['multiple_intro'] = movies['Cinemagoer_id'].apply(get_intro)

In [ ]:
# getting the synopsis from the Cinemagoer movie id
def get_synopsis(movie_id):
    if movie_id is not None:
        movie = ia.get_movie(movie_id, info=['synopsis'])
        if 'synopsis' in movie:
            print(movie['synopsis'])
            return movie['synopsis']
    else:
        return None

# Add the plots to the movies DataFrame
movies['synopsis'] = movies['Cinemagoer_id'].apply(get_synopsis)

In [ ]:
# getting the reviews from the Cinemagoer movie id
def get_reviews(movie_id):
    if movie_id is not None:
        movie = ia.get_movie(movie_id, info=['reviews'])
        if 'reviews' in movie:
            reviews = [review['content'] for review in movie['reviews']]
            print(reviews)
            return reviews
    else:
        return None

# Add the plots to the movies DataFrame
movies['reviews'] = movies['Cinemagoer_id'].apply(get_reviews)

In [ ]:
# discard the list warpper for the reviews, intro and synopsis
movies['reviews'] = [ast.literal_eval(review)[0] if isinstance(review, str) else review for review in movies['reviews']]
movies['synopsis'] = [ast.literal_eval(text)[0] if isinstance(text, str) else text for text in movies['synopsis']]
movies['multiple_intro'] = [ast.literal_eval(text)[0] if isinstance(text, str) else text for text in movies['multiple_intro']]

movies['rating_cat'] = pd.qcut(movies['rating'], q=3, labels=[0, 1, 2])
speakers = speakers[speakers['gender'] != '?']

In [ ]:
# put the dialogues together
df = pd.DataFrame(utterances)  # replace this with your actual DataFrame loading code

grouped = df.groupby('movie_idx').agg({'text': ' '.join,
                                       'utterances_id': list}).reset_index()
grouped = grouped.rename(columns={'text': 'conversation_data'})
grouped_utterances = grouped.rename(columns={'utterances_id': 'utterances_id_list'})
grouped

In [ ]:
movies.isna().sum()

In [ ]:
utterances.isna().sum()

In [ ]:
speakers.isna().sum()

In [ ]:
grouped.isna().sum()

In [ ]:
movies = movies.dropna().reset_index(drop=True)
utterances = utterances.dropna().reset_index(drop=True)
speakers = speakers.dropna().reset_index(drop=True)

In [6]:
speakers.to_csv('C:\\Users\\98768\\Desktop\\is310\\project\\movie-corpus\\processed\\speakers.csv', index=False)
movies.to_csv('C:\\Users\\98768\\Desktop\\is310\\project\\movie-corpus\\processed\\movies.csv', index=False)
utterances.to_csv('C:\\Users\\98768\\Desktop\\is310\\project\\movie-corpus\\processed\\utterances.csv', index=False)
grouped.to_csv('C:\\Users\\98768\\Desktop\\is310\\project\\movie-corpus\\processed\\grouped.csv', index=False)